In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import CWD, SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import resolve, Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from math import sqrt
from io import BytesIO
from itertools import permutations
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps

In [3]:
def create_grid(collection):
    row = len(collection)
    column = 1
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    border = offset = 20
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )

    return grid

In [4]:
def create_segment_grid(collection):
    length = len(collection)
    root = sqrt(length)
    root = int(root)
    
    column = row = root
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    offset = 10
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )
        
    return grid

In [5]:
def create_image(spectrogram):
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)

    return image

In [6]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [7]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [8]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [9]:
minimum = 10
maximum = minimum + 10

subset = dataframe.iloc[minimum:maximum]

In [10]:
segment = subset.segment.to_numpy()

In [11]:
import cv2
import scipy.ndimage

from scipy import ndimage as ndi
from skimage import color, data, feature, filters, img_as_float, morphology
from skimage.data import camera
from skimage.feature import peak_local_max
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint, unwrap_phase
from skimage.segmentation import chan_vese
from skimage.util import compare_images, random_noise

In [12]:
def process_image(image):
    image = cv2.medianBlur(image, 3)
    # image = cv2.fastNlMeansDenoising(image, None, 30, 7, 21)

    # Minimum
    minimum = image < 110
    image[np.where(minimum)] = 0

    # Maximum
    maximum = image > 180
    image[np.where(maximum)] = 255

    return image

In [13]:
%%capture

spectrogram = np.frompyfunc(
    lambda x, y: create_spectrogram(x, y),
    2,
    1
)(segment, settings)

In [14]:
original = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(spectrogram)

In [15]:
image = np.frompyfunc(
    lambda x: process_image(x),
    1,
    1
)(spectrogram)

In [16]:
image = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(image)

In [17]:
original_grid = create_segment_grid(original)
segment_grid = create_segment_grid(image)

collection = [original_grid, segment_grid]

grid = create_grid(collection)
grid.show()

E:\code\personal\warbler.py\venv\lib\site-packages\numpy\lib\function_base.py:2360: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  args = [asarray(arg) for arg in args]
E:\code\personal\warbler.py\venv\lib\site-packages\numpy\lib\function_base.py:2360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args = [asarray(arg) for arg in args]
E:\code\personal\warbler.py\venv\l